In [26]:
import importlib, os, gsw, gc

import pandas as pd
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from tqdm import tqdm

import SXBQ as sx

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib widget

importlib.reload(sx)


from scipy.fft import fft, fftfreq
from scipy import signal
import matplotlib.dates as mdates

# LOAD HYDROGRAPHIC DATA

In [2]:
data = sx.sxdf()
data.data = pd.read_parquet('C://Users/johan/OneDrive/Bureau/PFE-Goteborg/resultsBT.pqt')

top_mounted = False

def delimitateProfiles():
    #data.data.sort_values('Timestamp', ignore_index=True, inplace=True)
    data.median_resample()

    _gd = np.isfinite(data.data['diveNum'].values)
    _, _tmp = np.unique( np.round(data.data['diveNum'].values[_gd]) , return_inverse=True)

    data.data.loc[_gd,'diveNum'] = np.round(_tmp+1)
    data.data['diveNum'] = data.data['diveNum'].interpolate('nearest')
        
    data.data['profileNum'] = data.data['diveNum'].values*2
    _tmp = data.data['NAV_RESOURCE'].interpolate('nearest').values
    ind = (_tmp == 100) | (_tmp == 110) | (_tmp == 116)
    data.data.loc[ind,'profileNum'] = data.data.loc[ind,'profileNum'] - 1

delimitateProfiles()

def rmsd(x):
    return np.sqrt(np.nanmean(x**2))

#data.data

# LOAD ADCP DATA

In [3]:
#for second set of data
header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']

adcp_path1 = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA057_M28/ADCP/sea057_M28_burst_B0' 
ADCP1 = pd.read_table(adcp_path1+'.sen',sep=r"\s+",names=header) #Load the .sen file
ADCP1.insert(0,'time',pd.to_datetime(ADCP1[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP1.set_index('time', inplace=True)

ADCP1 = ADCP1.to_xarray()

for beam in tqdm(['1','2','3','4']):
    ADCP1['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path1+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )
#-------------------------------
header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']

adcp_path2 = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA057_M28/ADCP2/sea057_M28_burst_0000_B0' 
ADCP2 = pd.read_table(adcp_path2+'.sen',sep=r"\s+",names=header) #Load the .sen file
ADCP2.insert(0,'time',pd.to_datetime(ADCP2[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP2.set_index('time', inplace=True)

ADCP2 = ADCP2.to_xarray()

for beam in tqdm(['1','2','3','4']):
    ADCP2['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path2+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )
    
#-------------------------------
header = ['Month','Day','Year','Hour','Minute','Second','Burst counter','Ensemble counter','Error code','Status code','Battery voltage','Soundspeed','Heading','Pitch','Roll','Pressure','Temperature','Analog input 1','Analog input 2']

adcp_path3 = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA057_M28/ADCP3/sea057_M28_burst_0001_B0' 
ADCP3 = pd.read_table(adcp_path3+'.sen',sep=r"\s+",names=header) #Load the .sen file
ADCP3.insert(0,'time',pd.to_datetime(ADCP3[['Year', 'Month', 'Day', 'Hour', 'Minute','Second']], utc=True, origin='unix', cache='False'))

ADCP3.set_index('time', inplace=True)

ADCP3 = ADCP3.to_xarray()

for beam in tqdm(['1','2','3','4']):
    ADCP3['V'+beam] = (
                   ['time','bin'],
                   pd.read_csv(adcp_path3+'.v'+beam, sep=r"\s+", header=None).iloc[:,2:]
                  )

#Utiliser pd.concat()  (https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:30<00:00, 22.55s/it]


In [4]:
ADCP = xr.merge([ADCP1,ADCP2,ADCP3])
print(ADCP)

<xarray.Dataset>
Dimensions:           (bin: 60, time: 2586948)
Coordinates:
  * time              (time) object 1616497953001000000 ... 1617793088750000000
Dimensions without coordinates: bin
Data variables: (12/23)
    Month             (time) float64 3.0 3.0 3.0 3.0 3.0 ... 4.0 4.0 4.0 4.0 4.0
    Day               (time) float64 23.0 23.0 23.0 23.0 ... 7.0 7.0 7.0 7.0
    Year              (time) float64 2.021e+03 2.021e+03 ... 2.021e+03 2.021e+03
    Hour              (time) float64 11.0 11.0 11.0 11.0 ... 10.0 10.0 10.0 10.0
    Minute            (time) float64 12.0 12.0 12.0 12.0 ... 58.0 58.0 58.0 58.0
    Second            (time) float64 33.0 33.25 33.5 33.75 ... 8.25 8.5 8.75
    ...                ...
    Analog input 1    (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Analog input 2    (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    V1                (time, bin) float64 -1.135 0.525 2.538 ... 1.205 2.346
    V2                (time, bin) float

# LOAD BOTTOM TRACKING DATA

In [196]:
#for Bottom Tracking data
bt_path = 'C://Users/johan/OneDrive/Bureau/PFE-Goteborg/Scripts/Data/SEA057_M28/ADCP/telemetryfile'

header = ['Type','Beam','Date','Hour_Min_Sec','DT1','DT2','Bottom Velocity','Figure of Merit','Distance','Water Velocity','Status']
Telemetry = pd.read_table(bt_path+'.bin',sep=',',names=header) #To select only the first 1000 rows, use : nrows=1000
print("Data reading : OK")

BT = Telemetry[Telemetry['Type'] == '$PNORBT']      #Only keeps bottom tracking data from telemetry
BT = BT[BT['Distance'] > 6]                         #Only keep samples in range (distance not too far)
BT = BT[BT['Distance'] < 35]                        #Only keep samples in range (distance not too close)
BT = BT[BT['Bottom Velocity'] < 2.5]                #Only keep speed that are in range 
BT = BT[BT['Bottom Velocity'] > -2.5]                #Only keep speed that are in range



#Set time as Index
BT = BT.astype({"Hour_Min_Sec": 'str', "Date": 'str'}) #Convert time and date as a string
BT['Hour_Min_Sec']=pd.to_datetime(BT['Hour_Min_Sec'], format="%H%M%S.%f", errors='coerce').dt.time #Convert to Pandas time format
BT['Date']=pd.to_datetime(BT['Date'], format="%m%d%y", errors='coerce') #Convert to Pandas date format
BT = BT[BT['Hour_Min_Sec'].notnull()] #Check for errors in time
BT = BT[BT['Date'].notnull()] #Check for errors in date
print("All NaT values have been deleted from date and time")
BT.insert(0,'time',pd.to_datetime(BT['Date'].astype(str) + ' ' + BT['Hour_Min_Sec'].astype(str), utc=True, origin='unix', cache='False'))
BT.set_index('time', inplace=True)
if (~BT.index.is_monotonic) :
    print("Some data were not sorted by date...")
    BT=BT.sort_index() #Some data were not sorted by date
    if (BT.index.is_monotonic) :
        print("     Data is now sorted by date")





#Put Beam 1-4 on a single row
BT_1 = BT[BT['Beam'] ==1]
BT_2 = BT[BT['Beam'] ==2]
BT_3 = BT[BT['Beam'] ==3]
BT_4 = BT[BT['Beam'] ==4]

if not(BT_1.index.is_unique and BT_2.index.is_unique and BT_3.index.is_unique and BT_4.index.is_unique) :
    print('Duplicate index detected...')
    BT_1 = BT_1[~BT_1.index.duplicated(keep='first')]
    BT_2 = BT_2[~BT_2.index.duplicated(keep='first')]
    BT_3 = BT_3[~BT_3.index.duplicated(keep='first')]
    BT_4 = BT_4[~BT_4.index.duplicated(keep='first')]
    print('     Duplicate index deleted')
    

BT = BT_1 #Put all 4 beam on one row

if not(BT_1.columns.is_unique and BT_2.columns.is_unique and BT_3.columns.is_unique and BT_4.columns.is_unique) :
    print('Duplicate column detected')

BT.insert(0,'V1',BT_1['Bottom Velocity'])
BT.insert(0,'D1',BT_1['Distance'])
BT.insert(0,'FoM1',BT_1['Figure of Merit'])

BT.insert(0,'V2',BT_2['Bottom Velocity'])
BT.insert(0,'D2',BT_2['Distance'])
BT.insert(0,'FoM2',BT_2['Figure of Merit'])

BT.insert(0,'V3',BT_3['Bottom Velocity'])
BT.insert(0,'D3',BT_3['Distance'])
BT.insert(0,'FoM3',BT_3['Figure of Merit'])

BT.insert(0,'V4',BT_4['Bottom Velocity'])
BT.insert(0,'D4',BT_4['Distance'])
BT.insert(0,'FoM4',BT_4['Figure of Merit'])

#print(BT_1['Bottom Velocity'])
print(BT)

Data reading : OK
All NaT values have been deleted from date and time
Some data were not sorted by date...
     Data is now sorted by date
Duplicate index detected...
     Duplicate index deleted
                                 FoM4     D4       V4 FoM3     D3       V3  \
time                                                                         
2021-03-23 11:28:33.058800+00:00  NaN    NaN      NaN  NaN    NaN      NaN   
2021-03-23 11:28:33.308800+00:00  NaN    NaN      NaN  NaN    NaN      NaN   
2021-03-23 11:28:33.558800+00:00  NaN    NaN      NaN  NaN    NaN      NaN   
2021-03-23 11:28:33.808800+00:00  NaN    NaN      NaN  NaN    NaN      NaN   
2021-03-23 11:28:34.058800+00:00  NaN    NaN      NaN  NaN    NaN      NaN   
...                               ...    ...      ...  ...    ...      ...   
2021-04-07 22:44:07.058900+00:00  0.9  14.85  0.07551  0.8  13.73  0.14998   
2021-04-07 22:44:07.308900+00:00  1.0  15.01  0.07434  1.4  13.79  0.14998   
2021-04-07 23:08:56.5589

In [197]:
# Add roll pitch and heading to Bottom tracking data
A = BT.to_xarray()

#Insert Pitch
ADCP_Pitch = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Pitch'])]).T , columns=['time', 'Pitch']  )
#print(ADCP_Pitch)
ADCP_Pitch['time']=pd.to_datetime(ADCP_Pitch['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Pitch)
ADCP_Pitch.set_index('time', inplace=True)
#print(ADCP_Pitch)
BT.insert(0,'Pitch', ADCP_Pitch)

#Insert Roll
ADCP_Roll = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Roll'])]).T , columns=['time', 'Roll']  )
#print(ADCP_Roll)
ADCP_Roll['time']=pd.to_datetime(ADCP_Roll['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Roll)
ADCP_Roll.set_index('time', inplace=True)
#print(ADCP_Roll)
BT.insert(0,'Roll', ADCP_Roll)

#Insert Heading
ADCP_Heading = pd.DataFrame(   np.array([A['time'], np.interp(A['time'], ADCP['time'], ADCP['Heading'])]).T , columns=['time', 'Heading']  )
#print(ADCP_Heading)
ADCP_Heading['time']=pd.to_datetime(ADCP_Heading['time']/(10**3), unit='us', utc=True, origin='unix', cache='False')
#print(ADCP_Heading)
ADCP_Heading.set_index('time', inplace=True)
#print(ADCP_Heading)
BT.insert(0,'Heading', ADCP_Heading)

BT = BT.astype({"Pitch": 'float64', "Roll": 'float64', "Heading": 'float64',"Figure of Merit": 'float64'}) #Convert Pitch, Roll & Heading from string to float

BT = BT.to_xarray()
print(BT)

<xarray.Dataset>
Dimensions:          (time: 218267)
Coordinates:
  * time             (time) object 1616498913058800000 ... 1617838505558900000
Data variables: (12/26)
    Heading          (time) float64 268.3 268.3 268.3 ... 258.6 258.6 258.6
    Roll             (time) float64 3.5 3.5 3.5 3.851 5.0 ... 2.0 2.0 2.0 2.0
    Pitch            (time) float64 56.5 56.5 56.5 56.85 ... -2.9 -2.9 -2.9 -2.9
    FoM4             (time) object nan nan nan nan nan ... '1.0' nan nan nan
    D4               (time) float64 nan nan nan nan nan ... 15.01 nan nan nan
    V4               (time) float64 nan nan nan nan nan ... 0.07434 nan nan nan
    ...               ...
    DT2              (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Bottom Velocity  (time) float64 -0.5371 -0.5212 -0.4462 ... -0.2355 -0.2314
    Figure of Merit  (time) float64 18.3 18.5 11.7 19.4 ... 25.6 11.2 8.7 10.5
    Distance         (time) float64 20.86 19.03 18.75 ... 14.11 12.28 12.17
    Water Velocity  

# CALCULATE ADCP BIN DEPTH

In [9]:
def getADCPBinDepth(ADCP, bin_size, blanking_distance,direction='down',lat=58):
    # Retuns a new coordinate within the ADCP matrix of size ntime x nbin containing ADCP bin depths
    if top_mounted:
        direction = 1
    else:
        direction = -1

    bin_distance = blanking_distance + np.arange(len(ADCP.bin))*bin_size + 0.5*bin_size
    
    alpha = np.arccos(
                np.cos(np.deg2rad(22.5 + ADCP['Pitch'])) * # THIS IS WRONG
                np.cos(np.deg2rad(ADCP['Roll'])) 
            )
    adcp_depth = gsw.z_from_p(ADCP['Pressure'],lat)

    bin_depth = np.tile(adcp_depth, (len(ADCP.bin), 1)).T \
                - direction \
                * np.tile(bin_distance, (len(ADCP.time), 1)) \
                * np.tile(np.cos(alpha), (len(ADCP.bin), 1)).T \

    return ADCP.assign_coords({'bin_depth':(['time','bin'], bin_depth)})

ADCP = getADCPBinDepth(ADCP,0.5,0.2,55)

# SOUNDSPEED CORRECTION

In [10]:
# Not implemented yet

print(ADCP['Soundspeed'])

<xarray.DataArray 'Soundspeed' (time: 2586948)>
array([1490.3, 1490.3, 1490.3, ..., 1480.3, 1480.3, 1480.3])
Coordinates:
  * time     (time) object 1616497953001000000 ... 1617793088750000000


# Rotation correction

In [11]:
# Not implemented yet

# Bottom Tracking (3 beam pitch-dependent solution to XYZ and ENU calculation)

In [198]:
def calcXYZfrom3beam():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam_fore = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    xyz2beam_aft = np.array([
        [-sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz_fore = np.linalg.inv(xyz2beam_fore)
    beam2xyz_aft = np.linalg.inv(xyz2beam_aft)


    V_fore = beam2xyz_fore @ np.array([
        BT['V1'].values.flatten(),
        BT['V2'].values.flatten(),
        BT['V4'].values.flatten()
        ])
    V_aft = beam2xyz_aft @ np.array([
        BT['V3'].values.flatten(),
        BT['V2'].values.flatten(),
        BT['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')


    X_fore = np.reshape( V_fore[0,:] , (-1,1) ) #Because number of cell is 1
    X_aft = np.reshape( V_aft[0,:] , (-1,1) ) #Because number of cell is 1
    Y_fore = np.reshape( V_fore[1,:] , (-1,1) ) #Because number of cell is 1
    Y_aft = np.reshape( V_aft[1,:] , (-1,1) ) #Because number of cell is 1
    Z_fore = np.reshape( V_fore[2,:] , (-1,1) ) #Because number of cell is 1
    Z_aft = np.reshape( V_aft[2,:] , (-1,1) ) #Because number of cell is 1
    
    use_aft_on_climb = BT['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X_fore[~use_aft_on_climb,:] = X_aft[~use_aft_on_climb,:]
        Y_fore[~use_aft_on_climb,:] = Y_aft[~use_aft_on_climb,:]
        Z_fore[~use_aft_on_climb,:] = Z_aft[~use_aft_on_climb,:]
    else:
        print('Assuming ADCP is bottom mounted')
        X_fore[use_aft_on_climb,:] = X_aft[use_aft_on_climb,:]
        Y_fore[use_aft_on_climb,:] = Y_aft[use_aft_on_climb,:]
        Z_fore[use_aft_on_climb,:] = Z_aft[use_aft_on_climb,:]
        Y_fore=-Y_fore
        Z_fore=-Z_fore

    BT['X'] = (['time','bin'], X_fore )
    BT['Y'] = (['time','bin'], Y_fore ) 
    BT['Z'] = (['time','bin'], Z_fore )
    
    plt.close('all')
    _ = plt.hist(X_fore.flatten(),100,color='r',alpha=0.3)
    _ = plt.hist(X_aft.flatten(),100,color='b',alpha=0.3)
    
    
calcXYZfrom3beam()
#gc.collect()

Assuming ADCP is bottom mounted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [199]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = BT['Heading'].values
    P = BT['Pitch'].values
    R = BT['Roll'].values

    E = BT['X'].values.copy()
    N = BT['Y'].values.copy()
    U = BT['Z'].values.copy()


    r = len(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        E[i], N[i], U[i] = XYZ2ENU @ [E[i], N[i], U[i]]

    BT['E'] =  (['time','bin'], E )
    BT['N'] =  (['time','bin'], N )
    BT['U'] =  (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|███████████████████████████████████████████████████████████████████████| 218267/218267 [00:07<00:00, 30173.17it/s]


42

# ADCP (3 beam pitch-dependent solution to XYZ and ENU calculation)

In [15]:
#Old one
def calcXYZfrom3beam_Old():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    beam2xyz = np.linalg.inv(xyz2beam)

    V_fore = beam2xyz @ np.array([
        ADCP['V1'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])
    V_aft = beam2xyz @ np.array([
        ADCP['V3'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')

    X = np.reshape( V_fore[0,:] , (-1,60) ) #Because number of cell is 60
    X2 = np.reshape( V_aft[0,:] , (-1,60) ) #Because number of cell is 60
    
    plt.close('all')
    _ = plt.hist(X.flatten(),100,color='r',alpha=0.3)
    _ = plt.hist(-X2.flatten(),100,color='b',alpha=0.3)
    
    use_aft_on_climb = ADCP['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X[~use_aft_on_climb,:] = -X2[~use_aft_on_climb,:]
        V_aft[1,:] = -V_aft[1,:]
    else:
        print('Assuming ADCP is bottom mounted')
        X[use_aft_on_climb,:] = -X2[use_aft_on_climb,:]
    
    
    _ = plt.hist(X.flatten(),100,color='g',alpha=0.3)
    
    ADCP['X'] = (['time','bin'], X )
    ADCP['Y'] = (['time','bin'], np.reshape( V_aft[1,:] , (-1,60) ) ) #Because number of cell is 60
    ADCP['Z'] = (['time','bin'], np.reshape( V_aft[2,:] , (-1,60) ) ) #Because number of cell is 60
    
    
#calcXYZfrom3beam_Old()
#gc.collect()

In [16]:
def calcXYZfrom3beam_bis():
    def sin(x):
        return np.sin(np.deg2rad(x))
    def cos(x):
        return np.cos(np.deg2rad(x))

    a = 47.5 # Beam 1 and 3 angle from Z
    b = 25 # Beam 2 and 4 angle from Z

    xyz2beam_fore = np.array([
        [sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])
    
    xyz2beam_aft = np.array([
        [-sin(a),0,cos(a)],
        [0,-sin(b),cos(b)],
        [0,sin(b),cos(b)]
    ])

    beam2xyz_fore = np.linalg.inv(xyz2beam_fore)
    beam2xyz_aft = np.linalg.inv(xyz2beam_aft)

    V_fore = beam2xyz_fore @ np.array([
        ADCP['V1'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])
    V_aft = beam2xyz_aft @ np.array([
        ADCP['V3'].values.flatten(),
        ADCP['V2'].values.flatten(),
        ADCP['V4'].values.flatten()
        ])

    if rmsd(V_aft[1:,:]-V_fore[1:,:]) != 0:
        print('Something is wrong - abort and investigate...')

    X_fore = np.reshape( V_fore[0,:] , (-1,60) ) #Because number of cell is 60
    X_aft = np.reshape( V_aft[0,:] , (-1,60) ) #Because number of cell is 60
    Y_fore = np.reshape( V_fore[1,:] , (-1,60) ) #Because number of cell is 60
    Y_aft = np.reshape( V_aft[1,:] , (-1,60) ) #Because number of cell is 60
    Z_fore = np.reshape( V_fore[2,:] , (-1,60) ) #Because number of cell is 60
    Z_aft = np.reshape( V_aft[2,:] , (-1,60) ) #Because number of cell is 60
    
    
    use_aft_on_climb = ADCP['Pitch'] > 0
    
    if top_mounted == True:
        print('Assuming ADCP is top mounted')
        X_fore[~use_aft_on_climb,:] = X_aft[~use_aft_on_climb,:]
        Y_fore[~use_aft_on_climb,:] = Y_aft[~use_aft_on_climb,:]
        Z_fore[~use_aft_on_climb,:] = Z_aft[~use_aft_on_climb,:]
        
    else:
        print('Assuming ADCP is bottom mounted')
        X_fore[use_aft_on_climb,:] = X_aft[use_aft_on_climb,:]
        Y_fore[use_aft_on_climb,:] = Y_aft[use_aft_on_climb,:]
        Z_fore[use_aft_on_climb,:] = Z_aft[use_aft_on_climb,:]
        Y_fore=-Y_fore
        Z_fore=-Z_fore

    ADCP['X'] = (['time','bin'], X_fore )
    ADCP['Y'] = (['time','bin'], Y_fore ) 
    ADCP['Z'] = (['time','bin'], Z_fore ) 
    
    plt.close('all')
    _ = plt.hist(X_fore.flatten(),100,color='g',alpha=0.3)
    
calcXYZfrom3beam_bis()
#gc.collect()

Assuming ADCP is bottom mounted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
def calcENUfromXYZ():
    def M_xyz2enu(heading,pitch,roll):
        hh = np.pi*(heading-90)/180
        pp = np.pi*pitch/180
        rr = np.pi*roll/180
        
        _H = np.array([
            [np.cos(hh),np.sin(hh),0], 
            [-np.sin(hh),np.cos(hh),0], 
            [0,0,1]
        ])
        _P = np.array([
            [np.cos(pp), -np.sin(pp)*np.sin(rr), -np.cos(rr)*np.sin(pp)] ,
            [0, np.cos(rr), -np.sin(rr)] , 
            [ np.sin(pp), np.sin(rr)*np.cos(pp), np.cos(pp)*np.cos(rr)]
        ])
        
        _R = _H@_P
        return _R

    H = ADCP['Heading'].values
    P = ADCP['Pitch'].values
    R = ADCP['Roll'].values

    E = ADCP['X'].values.copy()
    N = ADCP['Y'].values.copy()
    U = ADCP['Z'].values.copy()

    r,c = np.shape(E)

    for i in tqdm(range(r)):
        XYZ2ENU = M_xyz2enu(H[i],P[i],R[i])
        for j in range(c):
            E[i,j], N[i,j], U[i,j] = XYZ2ENU @ [E[i,j], N[i,j], U[i,j]]

    ADCP['E'] = (['time','bin'], E )
    ADCP['N'] = (['time','bin'], N )
    ADCP['U'] = (['time','bin'], U )

calcENUfromXYZ()
gc.collect()

100%|██████████████████████████████████████████████████████████████████████| 2586948/2586948 [11:30<00:00, 3744.66it/s]


65

In [18]:
plt.figure()

PD = ADCP['Pitch'] < 0
PU = ADCP['Pitch'] > 0

plt.subplot(511)
_ = plt.hist(ADCP.isel(bin=0)['X'][PD].values,np.linspace(-1,1,100),color='r')
_ = plt.hist(ADCP.isel(bin=0)['X'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider moving forward so expect X negative')

plt.subplot(513)
_ = plt.hist(ADCP.isel(bin=0)['U'][PD].values,np.linspace(-1,1,100),color='r')
plt.title('Glider diving so expect U positive')

plt.subplot(515)
_ = plt.hist(ADCP.isel(bin=0)['U'][PU].values,np.linspace(-1,1,100),color='b')
plt.title('Glider climbing so expect U negative')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Glider climbing so expect U negative')

In [255]:
def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')

%matplotlib widget
plt.close('all')

ADCP_spd_xz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2  + ADCP.isel(bin=0)['Z']**2)
BT_spd_xz = np.sqrt(BT.isel(bin=0)['X']**2 + BT.isel(bin=0)['Y']**2 + BT.isel(bin=0)['Z']**2)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
#plt.plot(data.data['Timestamp'].values.astype('float')/(10**9), data.data.speed,'--k', alpha=0.5,label="steady flight model")
plt.plot(ADCP.time.values.astype('float')/(10**9),smooth(ADCP_spd_xz,100), alpha=0.7, color='g',label="ADCP")
plt.plot(BT.time.values.astype('float')/(10**9),smooth(BT_spd_xz,5), alpha=0.7, color='r',label="Bottom Tracking")
plt.title("speed of glider")
plt.xlabel("t")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float')/(10**9),[0,100])

def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([0,0.6])
step = np.diff(t)/100
update(35)


print((t[0] + 31*step))
print((t[0] + 31*step + step))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1.61689834e+09]
[1.61691126e+09]


# Fast Fourier Transform of ADCP signal

In [221]:
#Compute average dive time for the entire flight ---------------------------------------------------------------
diveNum= np.asarray(  data.data['diveNum'].values  )
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )
index=np.zeros( (diveNum[-1],) , dtype=int)
for i in range(diveNum[-1]) :
    index[i] = np.argmax(diveNum==i+1)
for i in range(diveNum[-1]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : ")
print(T_dive)
f_dive=1/T_dive
print("Average dive frequency [Hz] : ")
print(f_dive)


#Compute average dive time for the interval considered ---------------------------------------------------------------

x=30#Use this variable to chose the interval


time=sx.date2float(data.data['Timestamp'])
t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

diveNum= np.asarray(  data.data['diveNum'].values  )[i1:i2]
diveTime= np.asarray(  sx.date2float(data.data['Timestamp']).values  )[i1:i2]
index=np.zeros( (diveNum[-1]-diveNum[0],) , dtype=int)
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = np.argmax(diveNum==i+diveNum[0])
for i in range(diveNum[-1]-diveNum[0]) :
    index[i] = int(diveTime[index[i]])
T_dive=np.mean(np.diff(index))
print("Average dive period [s] : " + str(T_dive))
f_dive=1/T_dive
print("Average dive frequency [Hz] : " + str(f_dive))
print()
#----------------------------------------------------------------------------------------------------------------------



#Compute FFT of ADCP -------------------------------------------------------------------------------------------------------------
ADCP_Time=ADCP.time.values.astype('float')/(10**9)
ADCP_spd_xyz = np.asarray(  np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)  )


#Interval to study
t = np.nanpercentile(ADCP_Time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(ADCP_Time - x1)).argmin()
i2=(np.abs(ADCP_Time - x2)).argmin()



#Raw data
N = len(ADCP_Time[i1:i2])
T = 0.25            #Sampling period in sec
x = ADCP_Time[i1:i2]
y = ADCP_spd_xyz[i1:i2]


#smooth data----------------------
f_cutoff=0.014 #Hz
sos = signal.butter(1, f_cutoff, 'lowpass', fs=1/T, output='sos') #order, cutoff frequency, "lowpass", sampling frequency,
y = signal.sosfilt(sos, y)
print("Cutoff frequency : " + str(f_cutoff) + " [Hz]")


#Delete the last data because absurd result in time--------------------------
#N = len(ADCP_Time[0:-10])
#T = 10
#x = ADCP_Time[0:-10]
#y = ADCP_spd_xyz[0:-10]


#Delete the constant part of speed----------------------------
print("Average ADCP speed [m/s] :")
print(np.mean(y))
y=y-np.mean(y)

#Sampled data
#T = 50
#x=x[::5]
#y=y[::5]
#N = len(x)


yf = fft(y)
xf = fftfreq(N, T)[:N//2]






#Plotting figure ------------------------------------------------------------------------
plt.close('all')
fig = plt.figure()


ax0 = fig.add_subplot(2, 1, 1)
T=pd.to_datetime(x, unit='s', utc=True, origin='unix', cache='False')
plt.plot(T[25:], y[25:])
plt.title('Time varying component of ADCP speed')
if (T[0].strftime('%d %B %Y')==T[-1].strftime('%d %B %Y')) :
    plt.xlabel(T[0].strftime('%d %B %Y'))
else :
    plt.xlabel(T[0].strftime('%d %B %Y') + ' - ' + T[-1].strftime('%d %B %Y'))
plt.ylabel('Speed [m/s]')
ax0.set_ylim([-0.25,0.35])
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.grid()

ax = fig.add_subplot(2, 1, 2)
plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]), label="FFT(ADCP speed)")
#plt.plot([f_dive,f_dive], [-1,1], label="Average dive frequency")
plt.xlabel('Frequency [Hz]')
plt.title('FFT(Time varying component of ADCP speed)')
#ax.set_ylim([-0.01,0.05])
ax.set_xscale('log')
#ax.set_yscale('log')
#ax.set_xlim([0,0.01])
plt.grid()


plt.legend()
plt.tight_layout()
plt.show()

Average dive period [s] : 
3539.8406593406594
Average dive frequency [Hz] : 
0.0002824985913875182
Average dive period [s] : 2460.5
Average dive frequency [Hz] : 0.000406421459053038

Cutoff frequency : 0.014 [Hz]
Average ADCP speed [m/s] :
0.28537136187480244


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [218]:
#Compute FFT of Pressure -------------------------------------------------------------------------------------------------------------
time=sx.date2float(data.data['Timestamp'])
pressure=fillGaps(time, data.data.pressure)
dZdt =fillGaps(time, np.gradient(Depth,time))

#Interval to study
x=30

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



#Raw data
N = len(time[i1:i2])
T = 1            #Sampling period in sec
x = time[i1:i2]
y = dZdt[i1:i2]


#smooth data----------------------
f_cutoff=0.014 #Hz
sos2 = signal.butter(1, f_cutoff, 'lowpass', fs=1/T, output='sos') #order, cutoff frequency, "lowpass", sampling frequency,
y = signal.sosfilt(sos2, y)
print("Cutoff frequency : " + str(f_cutoff) + " [Hz]")


#Delete the last data because absurd result in time--------------------------
#N = len(ADCP_Time[0:-10])
#T = 10
#x = ADCP_Time[0:-10]
#y = ADCP_spd_xyz[0:-10]


#Delete the constant part of speed----------------------------
print("Average ADCP speed [m/s] :")
print(np.mean(y))
y=y-np.mean(y)

#Sampled data
#T = 50
#x=x[::5]
#y=y[::5]
#N = len(x)


yf = fft(y)
xf = fftfreq(N, T)[:N//2]






#Plotting figure ------------------------------------------------------------------------
plt.close('all')
fig = plt.figure()


ax0 = fig.add_subplot(2, 1, 1)
T=pd.to_datetime(x, unit='s', utc=True, origin='unix', cache='False')
plt.plot(T[25:], y[25:])
plt.title('Time varying component of pressure sensor speed')
if (T[0].strftime('%d %B %Y')==T[-1].strftime('%d %B %Y')) :
    plt.xlabel(T[0].strftime('%d %B %Y'))
else :
    plt.xlabel(T[0].strftime('%d %B %Y') + ' - ' + T[-1].strftime('%d %B %Y'))
plt.ylabel('Speed [m/s]')
ax0.set_ylim([-0.25,0.35])
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.grid()

ax = fig.add_subplot(2, 1, 2)
plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]), label="FFT(ADCP speed)")
#plt.plot([f_dive,f_dive], [-1,1], label="Average dive frequency")
plt.xlabel('Frequency [Hz]')
plt.title('FFT(Time varying component of pressure sensor speed)')
#ax.set_ylim([-0.01,0.05])
ax.set_xscale('log')
#ax.set_yscale('log')
#ax.set_xlim([0,0.01])
plt.grid()


plt.legend()
plt.tight_layout()
plt.show()

Cutoff frequency : 0.014 [Hz]
Average ADCP speed [m/s] :
0.0066024729192118515


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Searching for errors

### Computation of useful variables for plotting (ex: spd error between Steady state VS ADCP)

In [248]:
#Useful functions
def rmsd(x):
    return np.sqrt(np.nanmean(x**2))
def smooth(x,N):
    return np.convolve(x, np.ones(N)/N, mode='same')
def fillGaps(x,y):
    f = interp1d(x[np.isfinite(x+y)],y[np.isfinite(x+y)], bounds_error=False, fill_value=np.NaN)
    return(f(x))



#Useful variables ----------------------------------------------------------------------------
time=sx.date2float(data.data['Timestamp'])

_u = np.remainder(data.data.profileNum,2) == 0
_d = np.remainder(data.data.profileNum,2) == 1

Temperature=fillGaps(time, data.data.Temperature)
pressure=fillGaps(time, data.data.pressure)
salinity=fillGaps(time, data.data.salinity)
Ballast=fillGaps(time, data.data.BallastPos/1000000)
lon=fillGaps(time, data.data.lon)
lat=fillGaps(time, data.data.lat)

Depth = gsw.z_from_p(pressure,lat) # m . Note depth (Z) is negative, so diving is negative dZdt      
dZdt =fillGaps(time, np.gradient(Depth,time))

SA = gsw.SA_from_SP(salinity, pressure, lon, lat)
CT = gsw.CT_from_t(SA, Temperature, pressure)
rho = gsw.rho(SA, CT, pressure)



# Interpolation of ADCP data in order to match with drone data (data.data) -----------------------------------------
smooth_coeff=50
ADCP_Time=ADCP.time.values.astype('float')/(10**9)

ADCP_spd_xyz = np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Y']**2 + ADCP.isel(bin=0)['Z']**2)
ADCP_speed=np.interp(time, ADCP_Time, ADCP_spd_xyz)

ADCP_xz_spd=np.interp(time, ADCP_Time, np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Z']**2) )

ADCP_vert_spd = np.interp(time, ADCP_Time, ADCP.isel(bin=0)['U'])

ADCP_x_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['X'])
ADCP_y_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Y'])
ADCP_z_spd=np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Z'])
ADCP_pitch=np.deg2rad(np.interp(time, ADCP_Time, ADCP.isel(bin=0)['Pitch']))
ADCP_hrz_spd =  (ADCP_x_spd)*np.cos(np.deg2rad(ADCP_pitch)) -(ADCP_z_spd)*np.sin(np.deg2rad(ADCP_pitch))

ADCP_pressure=np.interp(time, ADCP_Time, ADCP.Pressure.values.astype('float') )
ADCP_pitch=np.interp(time, ADCP_Time, ADCP.Pitch.values.astype('float') )



#Influence of drone rotation in ADCP measurement -------------------------------------------------------------------
omega = fillGaps(time, np.gradient(ADCP_pitch,time))





# Interpolation of Bottom Tracking data in order to match with drone data (data.data) --------------------------------------
BT_spd_xyz = np.sqrt(BT.isel(bin=0)['X']**2 + BT.isel(bin=0)['Y']**2 + BT.isel(bin=0)['Z']**2)
BT_Time=BT.time.values.astype('float')/(10**9)
#BT_spd_xyz = fillGaps(BT_spd_xyz,BT_Time)
BT_speed = np.interp(time, BT_Time, BT_spd_xyz)

BT_spd_xz = np.sqrt(BT.isel(bin=0)['X']**2 + BT.isel(bin=0)['Z']**2)
BT_xz_spd=np.interp(time, BT_Time, BT_spd_xz)

BT_vert_spd = np.interp(time, BT_Time, BT.isel(bin=0)['U'])

BT_x_spd=np.interp(time, BT_Time, BT.isel(bin=0)['X'])
BT_y_spd=np.interp(time, BT_Time, BT.isel(bin=0)['Y'])
BT_z_spd=np.interp(time, BT_Time, BT.isel(bin=0)['Z'])
BT_pitch=np.interp(time, BT_Time, BT['Pitch'])
BT_hrz_spd =  (BT_x_spd)*np.cos(np.deg2rad(BT_pitch)) - (BT_z_spd)*np.sin(np.deg2rad(BT_pitch))

# STEADY STATE 1

In [21]:
importlib.reload(sx)
    
def flight_model():              
    flight = sx.SlocumModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               mass=60.772) # Verify mass number !!  
    flight.model_function()
    data.data['speed_vert_notRegressed'] = flight.speed_vert   
    flight.regress()     
    data.data['alpha'] = flight.alpha
    data.data['speed'] = flight.speed
    data.data['speed_vert'] = flight.speed_vert
    data.data['speed_horz'] = flight.speed_horz
    data.data['w_H2O'] = flight.w_H2O
    data.data['_valid'] = flight._valid     
    return flight
       
flight_old = flight_model()

C:\Users\johan\OneDrive\Bureau\PFE-Goteborg\Scripts\SXBQ.py:341: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * _dynamic_pressure / self.rho / self.area_w)


Initial parameters:  {'mass': 60.772, 'vol0': 0.059015, 'area_w': 0.24, 'Cd_0': 0.044178749994477656, 'Cd_1': 1.1050612498618673, 'Cl': 2.7177974996602754, 'comp_p': 4.5e-06, 'comp_t': 6.5e-05}
Non-optimised score: 0.17312682966217174
Regressing...
Optimization terminated successfully.
         Current function value: 0.028015
         Iterations: 599
         Function evaluations: 964
Optimised parameters:  {'mass': 60.772, 'vol0': 0.05941084846811691, 'area_w': 0.24, 'Cd_0': 0.06844734664936589, 'Cd_1': 2.1016758849707404, 'Cl': 3.7551425484470276, 'comp_p': 3.45042126466767e-06, 'comp_t': 6.536432981429658e-05}
Final Optimised score: 0.028014852318881015


In [22]:
aoa=fillGaps(time, data.data.alpha)
Pitch=fillGaps(time, np.deg2rad(data.data.Pitch)) #DO NOT USE !!! Data is wrong when mass is moving
Roll=fillGaps(time, np.deg2rad(data.data.Roll))

speed=data.data.speed.values.astype('float')
vertical_speed =fillGaps(time, data.data.speed_vert)
vertical_speed_notRegressed =fillGaps(time, data.data.speed_vert_notRegressed)
horizontal_speed = speed*np.cos(aoa+Pitch)

#Acceleration criteria validity --------------------------------------------------------------------------------------
L = 2.7 #length of glider
V = np.nanmean(data.data.speed) #Average speed of drone
a_max = V*V/(10*L)
a = np.gradient(speed,time)

validity = np.sign(np.abs(a)-a_max)

#Speed error SS1
spd_err=ADCP_speed-speed

In [23]:
%matplotlib widget
plt.close('all')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(data.data['Timestamp'].values.astype('float'), data.data.speed,'--k', alpha=0.5,label="steady flight model")
plt.plot(data.data['Timestamp'].values.astype('float'), smooth(ADCP_speed,100), alpha=0.7, color='g',label="ADCP")
plt.title("speed of glider")
plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.legend()

t = np.nanpercentile(data.data['Timestamp'].values.astype('float'),[0,100])


def update(x=0):
    ax.set_xlim([t[0] + x*step , t[0] + x*step + step])
    fig.canvas.draw_idle()

ax.set_ylim([0,1.2])
step = np.diff(t)/100
update(31)


print((t[0] + 30*step)/10**9 )
print(t[0] + 30*step + step)
print(np.interp(1.59846038e+18, data.data['Timestamp'].values.astype('float'),  data.data.speed))
print(1.59847874e+18-1.59846038e+18)



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1.61688543e+09]
[1.61689834e+18]
nan
18360000000000.0


#### Plotting influence of different parameters (Temperature, Pressure, Salinity, Pitch, Alpha, ...)

In [24]:
x=300

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/1000
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

In [27]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

X=data.data['Timestamp'][i1:i2]
Y1=dZdt[i1:i2]
Y2=data.data.speed_vert_notRegressed[i1:i2]
Y3=data.data.speed_vert[i1:i2]


#X=X[data.data.Depth[i1:i2]<-10]
#Y1=Y1[data.data.Depth[i1:i2]<-10]
#Y2=Y2[data.data.Depth[i1:i2]<-10]
#Y3=Y3[data.data.Depth[i1:i2]<-10]

plt.plot(X,Y1,label='Pressure sensor')
plt.plot(X,Y3,label='Steady state (regressed R1)',linestyle='--')
plt.plot(X,Y2,label='Steady state (not regressed)',linestyle='--',color='brown')
plt.plot(X,Pitch[i1:i2])
plt.plot(X,ADCP_pitch[i1:i2]*np.pi/180)
plt.grid()
plt.title("Vertical glider speed")
plt.legend()
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.ylabel("speed [m/s]")

ax.set_ylim([-1,1])
fig.canvas.draw_idle()


xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sensor Comparison

#### Vertical Speed

In [ ]:
from itertools import groupby

x = np.array([10., 20., np.nan, 40., 50., np.nan, np.nan, np.nan, 10.,0.,-10.])
y = np.zeros_like(x, dtype=int)
y[np.where(np.isnan(x))] = 1 # Locate where the array is nan


z = []
for a, b in groupby(y, lambda x: x == 0) : 
    if a: # Where the value is 0, simply append to the list
        z.extend(list(b))
    else: # Where the value is one, replace 1 with the number of sequential 1's
         l = len(list(b))
         z.extend([l]*l)


In [253]:
x=31.5*2

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/200
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()



X2=pd.DataFrame( np.asarray(BT.time/(10**3)).T,columns=['Date'])
X2['Date']=pd.to_datetime(X2['Date'], unit='us', utc=True, origin='unix', cache='False')
X2['Timestamp']=X2['Date']
X2.set_index('Date', inplace= True)
X2=X2['Timestamp']
Y5=-np.asarray(BT.isel(bin=0)['U'])


#Filter
f_cutoff=0.028
sos = signal.butter(1, f_cutoff, 'lowpass', fs=1, output='sos') #order, cutoff frequency, "lowpass", sampling frequency,


#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(1, 1, 1)
X=data.data['Timestamp'][i1:i2]
Y1 = signal.sosfilt(sos, -ADCP_vert_spd[i1:i2])
Y2 = signal.sosfilt(sos,dZdt[i1:i2])
Y3 = -np.gradient(ADCP_pressure,time)[i1:i2]
Y4 = -BT_vert_spd[i1:i2]
plt.plot(X, Y1, alpha=0.5, label='ADCP')
plt.plot(X, Y2, alpha=0.5, label='Pressure sensor ')
#plt.plot(X, Y3, alpha=0.7, label='ADCP pressure sensor UP speed')
#plt.plot(X, Y4, alpha=1, label='BT UP speed')





C1=np.asarray(BT['FoM1'])
C2=np.asarray(BT['FoM2'])
C3=np.asarray(BT['FoM3'])
C4=np.asarray(BT['FoM4'])
C1=C1.astype('float')
C2=C2.astype('float')
C3=C3.astype('float')
C4=C4.astype('float')
C=np.asarray(BT['D1'])
#C=np.nan_to_num(C)
plt.plot(X2, Y5, alpha=1, color='r', label='Bottom Tracking',zorder=10)
#plt.scatter(X2, Y5, 3,c=C,cmap='plasma_r', alpha=1,zorder=30,label='BT UP speed')
#plt.clim([35,35.1])
#cbar = plt.colorbar(orientation="horizontal", pad=0.2)
#cbar.ax.set_title('Figure of Merit', rotation=0)


ax.set_ylim([-0.4,0.4])
ax.set_xlim([X[0],X[-1]])


plt.ylabel('speed [m/s]')
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.title('Vertical speed')


plt.grid()
plt.legend()
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
fig.canvas.draw_idle()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
#import sys
#np.set_printoptions(threshold=1000)
print(C)

[1616498197308900000 1616498197808900000 1616498227558900000 ...
 1617791267308900000 1617791267558900000 1617791267808900000]


#### Total Speed

In [254]:
x=31.5*2

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/200
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




X = data.data['Timestamp'][i1:i2]
Y = ADCP_speed[i1:i2]

X2=pd.DataFrame( np.asarray(BT.time/(10**3)).T,columns=['Date'])
X2['Date']=pd.to_datetime(X2['Date'], unit='us', utc=True, origin='unix', cache='False')
X2['Timestamp']=X2['Date']
X2.set_index('Date', inplace= True)
X2=X2['Timestamp']
Y2=np.asarray(BT_spd_xyz)

#Filter
f_cutoff=0.028
sos = signal.butter(1, f_cutoff, 'lowpass', fs=1, output='sos') #order, cutoff frequency, "lowpass", sampling frequency,
Y=signal.sosfilt(sos, Y)

#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


ax = fig.add_subplot(1, 1, 1)
plt.plot(X, Y, alpha=0.5, label='ADCP')

C1=np.asarray(BT['FoM1'])
C2=np.asarray(BT['FoM2'])
C3=np.asarray(BT['FoM3'])
C4=np.asarray(BT['FoM4'])
C1=C1.astype('float')
C2=C2.astype('float')
C3=C3.astype('float')
C4=C4.astype('float')
C=np.asarray(BT['D1'])
#C=np.nan_to_num(C)
plt.plot(X2, Y2, alpha=1, color='r', label='Bottom Tracking',zorder=10)
#plt.scatter(X2, Y5, 3,c=C,cmap='plasma_r', alpha=1,zorder=30,label='BT UP speed')
#plt.clim([35,35.1])
#cbar = plt.colorbar(orientation="horizontal", pad=0.2)
#cbar.ax.set_title('Figure of Merit', rotation=0)


ax.set_ylim([0,0.5])
ax.set_xlim([X[0],X[-1]])


plt.ylabel('speed [m/s]')
if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    plt.xlabel(X[0].strftime('%d %B %Y'))
else :
    plt.xlabel(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
plt.title('Total speed')


plt.grid()
plt.legend()
xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
fig.canvas.draw_idle()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Steady State with 2nd Speed constraint

In [31]:
import SteadyState2Constraints as ss2
importlib.reload(ss2)

flight = ss2.SteadyState_2_Model(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               ADCP_pitch, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               0.5, #Tau
               smooth(ADCP_speed,50),
               mass=60.772) # Verify mass number !!

C:\Users\johan\OneDrive\Bureau\PFE-Goteborg\Scripts\SteadyState2Constraints.py:283: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * _dynamic_pressure / self.rho / self.area_w)


In [32]:
flight.model_function()

In [33]:
flight.Apply_lowpass_filter()

In [ ]:
flight.regress()

In [34]:
x=30 #291

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()




#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()


x=30

t = np.nanpercentile(time,[0,100])
step = np.diff(t)/100
x1 = t[0] + x*step 
x2 = t[0] + x*step + step
i1=(np.abs(time - x1)).argmin()
i2=(np.abs(time - x2)).argmin()

#Plot vertical speed -----------------------------------------------------------------------------------------
ax3 = fig.add_subplot(2, 1, 1)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(dZdt,50)[i1:i2]
Y2=data.data.speed_vert[i1:i2]
Y3=flight.speed_vert[i1:i2]


plt.plot(X,Y1, alpha=1,label='Pressure sensor')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Vertical glider speed")
plt.ylabel("speed [m/s]")
ax3.set_ylim([-0.4,0.8])



#Plot total speed -----------------------------------------------------------------------------------------
ax4 = fig.add_subplot(2, 1, 2)

X=data.data['Timestamp'][i1:i2]
Y1=smooth(ADCP_speed,50)[i1:i2]
Y2=data.data.speed[i1:i2]
Y3=flight.speed[i1:i2]

plt.plot(X,Y1,label='ADCP')
plt.plot(X,Y2,label='Steady state (regressed with R1)',linestyle='--')
plt.plot(X,Y3,label='Steady state (regressed with R2)',linestyle='--',color='red')

plt.legend()
plt.grid()
plt.title("Total glider speed")
plt.ylabel("speed [m/s]")
ax4.set_ylim([-0.1,0.5])




fig.canvas.draw_idle()

if (X[0].strftime('%d %B %Y')==X[-1].strftime('%d %B %Y')) :
    fig.suptitle(X[0].strftime('%d %B %Y'))
else :
    fig.suptitle(X[0].strftime('%d %B %Y') + ' - ' + X[-1].strftime('%d %B %Y'))
    
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

xformatter = mdates.DateFormatter('%H:%M')
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.gcf().axes[1].xaxis.set_major_formatter(xformatter)
#plt.tight_layout()
plt.show()


print(i1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

387584


# Dynamic flight model -------------------------------------------------------------------------

In [ ]:
import dynamic_model as DM
importlib.reload(DM)


adcp_spd_xz = smooth(np.sqrt(ADCP.isel(bin=0)['X']**2 + ADCP.isel(bin=0)['Z']**2),100)
adcp_time = ADCP['time']/(10**9)


flight_dyn = DM.DynamicFlightModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               data.data.Pitch.interpolate('index').values, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               adcp_spd_xz,
               adcp_time)

flight_dyn.set_initial_conditions(0,0)

print(flight_dyn.t.size)

In [ ]:
#flight_dyn.solveRK4()
flight_dyn.regress()

#### Comparison of  TOTAL SPEED (Dyanmic VS Static VS ADCP)

In [ ]:
#print(flight_dyn.w[0:300])
plt.close('all')

pitch=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch)
dZdt=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.dZdt)
rho=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.rho)
pressure=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pressure)
ballast=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.ballast)
g=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.g)
temperature=np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.temperature)
alpha=np.arctan2(flight_dyn.w,flight_dyn.u)-pitch

Fb, Fg = flight_dyn.compute_FB_and_Fg(g, rho, pressure, ballast, temperature)
L, D=flight_dyn.compute_lift_and_drag(pitch, rho, flight_dyn.u, flight_dyn.w)



Fx=np.sin(pitch + alpha)*L-np.cos(pitch + alpha)*D
Fy=Fb - Fg -np.cos(pitch + alpha)*L -np.sin(pitch + alpha)*D



fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

#Dynamic model Speed
#plt.plot(flight_dyn.t, flight_dyn.u, 'r', label="u dyn [m/s]")
#plt.plot(flight_dyn.t, flight_dyn.w, 'b', label="w dyn [m/s]")
plt.plot(flight_dyn.t, np.sqrt(flight_dyn.w**2+flight_dyn.u**2), 'r', label="U dyn [m/s]")

#Dynamic model forces
#plt.plot(flight_dyn.t, pitch, 'k')
#plt.plot(flight_dyn.t, D/10, 'y')
#plt.plot(flight_dyn.t, L/10, 'y')

#Dynamic model pitch, depth, alpha, ...
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'g')
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.depth), 'k')
#plt.plot(flight_dyn.t, np.arctan2(flight_dyn.w,flight_dyn.u)-np.interp(flight_dyn.t, flight_dyn.time, flight_dyn.pitch), 'y')

#Steady state speed
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), data.data.speed_horz),'--m',label="u stat [m/s]")
#plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), data.data.speed_vert),'--c',label="w stat [m/s]")
plt.plot(flight_dyn.t, np.interp(flight_dyn.t, DM.date2float(data.data.Timestamp), data.data.speed), '--y',label="U stat [m/s]")

#ADCP speed
#plt.plot(time, smooth(ADCP_xz_spd,smooth_coeff), '--r',label="U ADCP [m/s]") #ADCP speed interpolated 
plt.scatter(ADCP.time.values.astype('float')/(10**9),smooth(adcp_spd_xz,100), 1,'b', alpha=1, label='ADCP speed') #ADCP speed not interpolated 



plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.title("Total speed comparison")
plt.legend()
ax.set_ylim([-0.5,1.1])
ax.set_xlim([flight_dyn.t[0],flight_dyn.t[-1]])
plt.show()


#### Comparison of speed error (relative to ADCP) between dynamic and static

In [ ]:
Dyn_Speed=np.interp(time, flight_dyn.t, np.sqrt(flight_dyn.w**2+flight_dyn.u**2))

# Definition of error function
Dyn_spd_err=smooth(ADCP_speed,smooth_coeff)-Dyn_Speed

#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(time, Dyn_spd_err, 1, 'r', alpha=1, label ='Dynamic speed error')
plt.scatter(time, spd_err, 1, 'b', alpha=1, label ='Static speed error')
plt.grid()
plt.legend()
plt.title("speed error")
plt.xlabel("t [s]")
plt.ylabel("speed error [m/s]")

plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.title("Speed error comparison")
plt.legend()
ax.set_ylim([-0.5,1.1])
ax.set_xlim([flight_dyn.t[0],flight_dyn.t[-1]])
plt.show()

#### Comparison of  HORIZONTAL SPEED (Dyanmic VS Static VS ADCP)

In [ ]:
#Figure plotting -----------------------------------------------------------------------------------
%matplotlib widget
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(time,horizontal_speed,'--y', alpha=0.6, label='Steady state Horizontal speed')
plt.plot(flight_dyn.t, flight_dyn.u, 'r',alpha=0.6, label="Dynamic Horizontal speed")
plt.scatter(time,smooth(-ADCP_hrz_spd,smooth_coeff), 1,'b', alpha=1, label='ADCP Horizontal speed') #Because : Vadcp = - Vdrone
plt.grid()
plt.legend()
plt.title("Horizontal speed")
plt.xlabel("t [s]")
plt.ylabel("speed error [m/s]")

plt.xlabel("t [s]")
plt.ylabel("v [m/s]")
plt.title("Horizontal speed comparison")
plt.legend()
ax.set_ylim([-0.5,1.1])
ax.set_xlim([flight_dyn.t[0],flight_dyn.t[-1]])
plt.show()